# Assignment 1 -- Cleaning CDC Cancer Data

**About** Assignment \#1 of CSS 577 Fall 2025

**Purpose** The purpose of this notebook is to clean cancer incidence data from the CDC so it will be easy to use for Assignment \#1. We will read in data from our data file, clean it up, and then save a new clean version of our data.

**Dataset**
We will be using the cancer incidence dataset, which includes the incidence of different cancers in the United States in the years 2019-2022. Note that incidence is different from prevalence. Incidence is the number of new cases, which prevalence is the total number of cases.

Source: https://wonder.cdc.gov/cancer-v2022.html

Last Accessed: September 2025

# 0. Import Statements and Copy-on-Write

In [1]:
import pandas as pd
from pathlib import Path

In [23]:
pd.set_option("mode.copy_on_write", True) # will be the default in pandas 3.0, best to enforce it

# 1. Read in the Data

In [3]:
cwd = Path().cwd() #should be our src folder, containing this notebook
project_folder = cwd.parent #should be our project folder, containing the src folder
dataset_path = Path('data/cancer_incidence_2019-2022.csv') #using a data folder makes life easier

file = project_folder / dataset_path #join the project folder with the dataset path

if not file.exists():
    raise FileNotFoundError(f"Dataset file not found: {file}")

cancer = pd.read_csv(file)

## 1.1 Quick Initial Inspection of the Data

In [4]:
cancer.columns #Get the column names

Index(['Notes', 'Cancer Sites', 'Cancer Sites Code', 'Year', 'Year Code',
       'Sex', 'Sex Code', 'Count', 'Population', 'Age-Adjusted Rate',
       'Age-Adjusted Rate Lower 95% Confidence Interval',
       'Age-Adjusted Rate Upper 95% Confidence Interval', 'Crude Rate',
       'Crude Rate Lower 95% Confidence Interval',
       'Crude Rate Upper 95% Confidence Interval'],
      dtype='object')

In [5]:
cancer.shape #We should have 5092 rows and 15 columns originally

(5092, 15)

Let's take a look at some of the data using head() to get the first few rows

In [6]:
cancer.head()

,Notes,Cancer Sites,Cancer Sites Code,Year,Year Code,Sex,Sex Code,Count,Population,Age-Adjusted Rate,Age-Adjusted Rate Lower 95% Confidence Interval,Age-Adjusted Rate Upper 95% Confidence Interval,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval
0,NaN,All Invasive Cancer Sites Combined,0,1999.0,1999.0,Female,F,631741,140394214,420.5,419.4,421.5,450.0,448.9,451.1
1,NaN,All Invasive Cancer Sites Combined,0,1999.0,1999.0,Male,M,672530,135067134,572.4,571.0,573.8,497.9,496.7,499.1
2,NaN,All Invasive Cancer Sites Combined,0,2000.0,2000.0,Female,F,641968,141865770,421.3,420.3,422.4,452.5,451.4,453.6
3,NaN,All Invasive Cancer Sites Combined,0,2000.0,2000.0,Male,M,690598,136692444,577.0,575.6,578.3,505.2,504.0,506.4
4,NaN,All Invasive Cancer Sites Combined,0,2001.0,2001.0,Female,F,662313,143603977,427.6,426.6,428.7,461.2,460.1,462.3


We can infer what some of these columns mean, but there are also some things we don't know:

- All the notes we see are NaN. What do the notes say?
- Cancer Site means where in the body the cancer was located.
- Each Cancer Site appears to be assigned a numeric Cancer Site Code for convenience
- Year is the year in which the observation is made (i.e., diagnosed)
- Year Code looks the same as the Year...? Will need to see what's happening hear
- Sex is the patient's biological set
- Sex Code looks like a one-letter abbreviate for the sex. Do we need both?
- Count is the number of new diagnoses of cancer at that site for that sex in that year
- Population is the number of people in the United States of that sex in that year (we should check if this is the same for all Cancer Sites)
- We don't know exactly what age-adjusted rate means. Seems like the incidence rate, but changed somehow to reflect age. (We haven't checked the documentation yet). Our dataset doesn't have information about ages.
- The age-adjusted rate seems to contrast with the crude rate, which we can guess is an un-adjusted rate of the same measurement
- We have four columns giving us confidence intervals around the rate measurements.

**Granularity**: What does each row represent? Each row or observation is characterized by (Cancer Site, Year, Sex). The remaining columns are codes and measurements.

## 1.2 Check if Missing Data
Are some columns missing some values for some observations? We know from .shape() that there are 5092 rows. 

In [7]:
cancer.isna().sum() #gives the number of missing values in each column.

Notes                                              5039
Cancer Sites                                         53
Cancer Sites Code                                    53
Year                                                 53
Year Code                                            53
Sex                                                  53
Sex Code                                             53
Count                                                53
Population                                           53
Age-Adjusted Rate                                    53
Age-Adjusted Rate Lower 95% Confidence Interval      53
Age-Adjusted Rate Upper 95% Confidence Interval      53
Crude Rate                                           53
Crude Rate Lower 95% Confidence Interval             53
Crude Rate Upper 95% Confidence Interval             53
dtype: int64

So every column has exactly 53 NaN values, except for notes which has 5039 NaN values. Notice that 53+5039 = 5092! Looks like whoever made this dataset likely added bunch of notes to the bottom or top of the document (53 rows of notes), and when we read them in, they're forming this separate column of Notes. But the rows in our dataset do not have their own notes.

We must be careful here -- it is too easy to get fooled into thinking that none of the 5039 rows with observations are missing data. We are not done addressing missing data.

## 1.3 Check that the Data Types make sense

Are the column data types what we expect? For some columns we should expect numeric types (int, float), while for others we do not.

In [8]:
cancer.dtypes

Notes                                               object
Cancer Sites                                        object
Cancer Sites Code                                   object
Year                                               float64
Year Code                                          float64
Sex                                                 object
Sex Code                                            object
Count                                               object
Population                                          object
Age-Adjusted Rate                                   object
Age-Adjusted Rate Lower 95% Confidence Interval     object
Age-Adjusted Rate Upper 95% Confidence Interval     object
Crude Rate                                          object
Crude Rate Lower 95% Confidence Interval            object
Crude Rate Upper 95% Confidence Interval            object
dtype: object

Here we see that only the year and Year Code are numbers, and they seem to be floats (not ints). Why aren't the other counter values and measurements like the count, population, rates, and confidence intervals numeric? There are a few possible reasons. Whoever created the dataset might be using some non-numeric characters or strings to represent missing data, or to annotate data points within the cells instead of creating separate columns for annotations. This could also be a sign that there is missing data, but instead of a NaN the missing data is being represented by some string. Let's take a look at the rows where some of the values are not numeric when we expect them to be.

Unfortunately, pandas doesn't have a nice method like .is_numeric() to just get the non-numeric values in a column. Instead we have to try to convert the column ot a numeric column, and then force all the rows where the conversion fails to be NaN values, then check which rows are NaN values.

In [9]:
non_numeric_count = pd.to_numeric(cancer['Count'], errors='coerce').isna()# Get a Boolean array (mask) for rows where 'Count' value does not convert to a number
cancer_non_numeric_count_rows = cancer[non_numeric_count]
cancer_non_numeric_count_rows

,Notes,Cancer Sites,Cancer Sites Code,Year,Year Code,Sex,Sex Code,Count,Population,Age-Adjusted Rate,Age-Adjusted Rate Lower 95% Confidence Interval,Age-Adjusted Rate Upper 95% Confidence Interval,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval
2497,NaN,Female Breast,26000-Female,1999.0,1999.0,Male,M,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
2499,NaN,Female Breast,26000-Female,2000.0,2000.0,Male,M,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
2501,NaN,Female Breast,26000-Female,2001.0,2001.0,Male,M,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
2503,NaN,Female Breast,26000-Female,2002.0,2002.0,Male,M,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
2505,NaN,Female Breast,26000-Female,2003.0,2003.0,Male,M,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5087,standard groupings of primary cancer sites.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5088,"6. For the 2005 year, the Census Bureau estima...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5089,Texas due to Hurricanes Katrina and Rita. CDC ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5090,"these states, nor are these counts included in...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1.4 Updating the File Reading

**NOTE**: In practice you would never add this second read here. Only read the data in once. You should update the code above and then re-run iteratively until you're done making sure the data is read in correctly. I'm reepeating the process below so that you get the experience of walking through the process above.

It looks like in our original dataset, there are some places where Count and other columns are Not Applicable instead of NaN. In particular, it looks like they create a separate Cancer Site Code for Females, instead of just using the Sex column, and there are Male rows for Female Breast cancer, where it is not applicable. The data isn't missing, it's just not applicable for that row. We might want to leave it this way, or we might want to make these values NaN. For our purposes, let's make these values NaN when we read in the file.

In [10]:
cancer = pd.read_csv(file, na_values=['Not Applicable']) #read in the file again, but this time make 'Not Applicable' into NaN
cancer.isna().sum()

Notes                                              5039
Cancer Sites                                         53
Cancer Sites Code                                    53
Year                                                 53
Year Code                                            53
Sex                                                  53
Sex Code                                             53
Count                                               461
Population                                          461
Age-Adjusted Rate                                   461
Age-Adjusted Rate Lower 95% Confidence Interval     461
Age-Adjusted Rate Upper 95% Confidence Interval     461
Crude Rate                                          461
Crude Rate Lower 95% Confidence Interval            461
Crude Rate Upper 95% Confidence Interval            461
dtype: int64

Now the data types should look better (float64 for numeric, object for text). However, some of the values are counted like 'Count', 'Population' and 'Year'. We will want to make these integers.

In [11]:
cancer.dtypes

Notes                                               object
Cancer Sites                                        object
Cancer Sites Code                                   object
Year                                               float64
Year Code                                          float64
Sex                                                 object
Sex Code                                            object
Count                                              float64
Population                                         float64
Age-Adjusted Rate                                  float64
Age-Adjusted Rate Lower 95% Confidence Interval    float64
Age-Adjusted Rate Upper 95% Confidence Interval    float64
Crude Rate                                         float64
Crude Rate Lower 95% Confidence Interval           float64
Crude Rate Upper 95% Confidence Interval           float64
dtype: object

Types should be float64 for measurements, Int64 for whole numbers, object for text

In [12]:
# Convert Year, Count, and Population columns to integers (nullable Int64 to handle NaNs)
cancer[['Year','Count','Population']] = cancer[['Year','Count','Population']].astype('Int64')
cancer.dtypes

Notes                                               object
Cancer Sites                                        object
Cancer Sites Code                                   object
Year                                                 Int64
Year Code                                          float64
Sex                                                 object
Sex Code                                            object
Count                                                Int64
Population                                           Int64
Age-Adjusted Rate                                  float64
Age-Adjusted Rate Lower 95% Confidence Interval    float64
Age-Adjusted Rate Upper 95% Confidence Interval    float64
Crude Rate                                         float64
Crude Rate Lower 95% Confidence Interval           float64
Crude Rate Upper 95% Confidence Interval           float64
dtype: object

**NOTE:** The column types may not reveal all missing values. For example, some people will us a flag values like 0, -1, or -999 to denote missing data. You must watch out for this. Check the documentation that comes with the data, and look at the min/max values in each column to see if they might be using numeric flag values for missing data.

# 2. Consistency Checks and Drops
Looking for errors, typos, redundancies, and other inconsistencies

- Is the Year always the same as the Year Code?
- Does the Sex Code always match Sex?
- Is the Cancer Site code always consistent for each Cancer Site?
- Is Count < Population for every observation?
- For each observational group (Cancer Site, Year, Sex), is the population the same for all Cancer Sites?

Also, do we need to keep all of our columns? Do we need all of our rows? Is there anything we would rather drop?

### 2.1 Redundancy Checks: Are the Code columns consistent? Does that make them redundant?

In [13]:
# Is the year always the same as the year code?
year_mismatch = cancer[cancer['Year'] != cancer['Year Code']]
year_mismatch.head()

,Notes,Cancer Sites,Cancer Sites Code,Year,Year Code,Sex,Sex Code,Count,Population,Age-Adjusted Rate,Age-Adjusted Rate Lower 95% Confidence Interval,Age-Adjusted Rate Upper 95% Confidence Interval,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval


In the cell above, we try to compare the Year to the Year Code and see if they are ever unequal. We use the != operator to check if they are unequal. However, in pandas, NaN != NaN. For our purposes, if both are NaN then we should say they are equal. But if one is NaN and the other is not, then we still want to see that as unequal. We can do this using the .equals() method.

In [14]:
#We can compare the columns using the .equals() method, which treats NaNs as equal by default.
cancer['Year'].equals(cancer['Year Code'])

False

We see that the Year Code column is entirely redundant. Now let's do a similar thing for Sex Code and Cancer Site Code to see if they are 100% consistent with Sex and Cancer Site, and decide if we really need both columns. Sometimes it's nice to have that redundancy, other times it is unnecessary.

In [15]:
# Check if Cancer Site Code is consistent for every Cancer Site
site_code_unique_counts = cancer.groupby('Cancer Sites')['Cancer Sites Code'].nunique()
inconsistent_site_codes = site_code_unique_counts[site_code_unique_counts > 1]
print("Number of Cancer Sites with inconsistent codes:", inconsistent_site_codes.size)
print("Cancer Sites with more than one code (if any):")
print(inconsistent_site_codes)

Number of Cancer Sites with inconsistent codes: 0
Cancer Sites with more than one code (if any):
Series([], Name: Cancer Sites Code, dtype: int64)


We can see that Year Code, Sex Code, and Cancer Site Code are consistent with their respective non-code variables, making them fully redundant. We can decide whether or not to keep these columns. Let's remove the Year Code column, keep the Sex Code instead of the Sex column and rename it to just Sex, and we'll keep both the Cancer Site and Cancer Site Code.

In [16]:
# Drop 'Year Code' and 'Sex' columns, and rename 'Sex Code' to 'Sex'
cancer = cancer.drop(columns=['Year Code', 'Sex'])
cancer = cancer.rename(columns={'Sex Code': 'Sex'})

### 2.2 Other Consistency Checks
- Count is never higher than population
- For each cancer site, each (Year, Sex) group should have the same population

First, let's check if the population numbers are the same (Year, Sex) combination for each Cancer Site. To do this we use groupby, to group according to these three columns, and then for eeach group we check whether there is more than one unique value in the population column for that group. I have broken it down into multiple steps so you can see what is happening.

In [17]:
cancer_grouped = cancer.groupby(['Cancer Sites', 'Year', 'Sex']) #this creates a groupby object, not a dataframe!
pop_unique_counts = cancer_grouped['Population'].nunique() #For each group, count the number of unique values in the Population column
print("Number of groups with more than one unique population value:", (pop_unique_counts > 1).sum())
print("Groups with more than one unique population value (if any):", pop_unique_counts[pop_unique_counts > 1])

Number of groups with more than one unique population value: 0
Groups with more than one unique population value (if any): Series([], Name: Population, dtype: int64)


In [18]:
# Check if Count is higher than Population for any rows
cancer[cancer['Count'] > cancer['Population']] #should be empty!

,Notes,Cancer Sites,Cancer Sites Code,Year,Sex,Count,Population,Age-Adjusted Rate,Age-Adjusted Rate Lower 95% Confidence Interval,Age-Adjusted Rate Upper 95% Confidence Interval,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval


Looking at the head() we have a Notes column but the values we see are all NaN. Let's look for any rows where these are not NaN, to see what information they give us. Looks like there are notes in some of the rows that are relevant -- they tell us how the data was created and how it is being represented. If we want to use this dataset we should read over the notes to understand them.

### 2.3 Notes Column

Let's finally deal with the Notes column that is mostly empty. We see that in all the rows where the Notes are not empty, there is no data.

In [19]:
cancer[cancer['Notes'].notna()].head()#look at the rows where Notes is not NaN

,Notes,Cancer Sites,Cancer Sites Code,Year,Sex,Count,Population,Age-Adjusted Rate,Age-Adjusted Rate Lower 95% Confidence Interval,Age-Adjusted Rate Upper 95% Confidence Interval,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval
5039,---,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
5040,Dataset: United States and Puerto Rico Cancer ...,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
5041,Query Parameters:,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
5042,Group By: Cancer Sites; Year; Sex,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
5043,Show Totals: Disabled,NaN,NaN,<NA>,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN


Wait, what is `<NA>`? Why are we suddenly seeing that? We know the column types are correct... This is another type of NaN for nullable integer types, which is what we made the Year column. `Nullable' just means that in can be a null like NaN or <NA>. Non-nullable variables have to hold a value, otherwise it throws an error.

We don't need the notes for our purposes, so let's just drop the notes column, and then remove all rows that do not contain any data

In [20]:
cancer = cancer.drop('Notes', axis=1) #drop rows where either Count or Population is NaN
cancer.dropna(inplace=True) #inplace=True modifies the dataframe directly instead of creating a copy

In [21]:
cancer.head() #Admire your beautiful clean data

,Cancer Sites,Cancer Sites Code,Year,Sex,Count,Population,Age-Adjusted Rate,Age-Adjusted Rate Lower 95% Confidence Interval,Age-Adjusted Rate Upper 95% Confidence Interval,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval
0,All Invasive Cancer Sites Combined,0,1999,F,631741,140394214,420.5,419.4,421.5,450.0,448.9,451.1
1,All Invasive Cancer Sites Combined,0,1999,M,672530,135067134,572.4,571.0,573.8,497.9,496.7,499.1
2,All Invasive Cancer Sites Combined,0,2000,F,641968,141865770,421.3,420.3,422.4,452.5,451.4,453.6
3,All Invasive Cancer Sites Combined,0,2000,M,690598,136692444,577.0,575.6,578.3,505.2,504.0,506.4
4,All Invasive Cancer Sites Combined,0,2001,F,662313,143603977,427.6,426.6,428.7,461.2,460.1,462.3


# 4. Save Clean Data

Now that our data is nice and clean, let's save it to a new data file so that we can use it in the future.

NOTE: We usually do not push the data files to our git repository, especially when using GitHub, unless the data set is small (like this one). GitHub is not made to store large datasets. We just include the link to download the data, and then include this notebook so that others can clean the data in the same way.

In [22]:
cancer.to_csv(project_folder / 'data/clean_cdc_cancer.csv', index=False) #index=False means don't write the index as a column in the csv file. We don't need it there.

**NOTE:** The method .to_csv overwrites files if they already exist, rather than appending data to the end of them